In [ ]:
import healpy as hp
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import haversine_distances

steps = 1000
order = 8
nside = hp.order2nside(order)
# Tile from polar region
lonlat = 0, np.degrees(np.arcsin(0.992))
tile_id = hp.ang2pix(nside, *lonlat, nest=True, lonlat=True)

fig = plt.figure(figsize=(16, 6))
ax_cyl = fig.add_subplot(1, 2, 1)
ax_pol = fig.add_subplot(1, 2, 2, projection='polar')

def plot(phi, z, **kwargs):
    theta = np.degrees(np.arccos(z))
    ax_cyl.plot(phi, z, **kwargs)
    ax_pol.plot(np.radians(phi), theta, **kwargs)

def hp_boundaries(tile_id, *, n=steps):
    hp_boundaries_xyz = hp.boundaries(nside, tile_id, nest=True, step=n)
    hp_boundaries_lonlat = hp.vec2ang(hp_boundaries_xyz.T, lonlat=True)
    phi = np.where(hp_boundaries_lonlat[0] < 180, hp_boundaries_lonlat[0], hp_boundaries_lonlat[0] - 360)
    return np.stack([phi, hp_boundaries_xyz[2]])

def hp_boundaries_side(tile_id, side_id, *, n=steps):
    hp_boundaries_xyz = hp.boundaries(nside, tile_id, nest=True, step=n)
    hp_boundaries_lonlat = hp.vec2ang(hp_boundaries_xyz.T, lonlat=True)
    phi = np.where(hp_boundaries_lonlat[0] < 180, hp_boundaries_lonlat[0], hp_boundaries_lonlat[0] - 360)
    return np.stack([phi[side_id*n:(side_id+1)*n], hp_boundaries_xyz[2][side_id*n:(side_id+1)*n]])

def hp_boundaries_side_latlon(tile_id, side_id, *, n=steps):
    hp_boundaries_xyz = hp.boundaries(nside, tile_id, nest=True, step=n)
    lon, lat = hp.vec2ang(hp_boundaries_xyz.T, lonlat=True)
    return np.stack([lat[side_id*n:(side_id+1)*n], lon[side_id*n:(side_id+1)*n]])

plot(*hp_boundaries_side(tile_id, 0), color='black', label='healpy')
plot(*hp_boundaries_side(tile_id, 1), color='red', label='healpy')
plot(*hp_boundaries_side(tile_id, 2), color='yellow', label='healpy')
plot(*hp_boundaries_side(tile_id, 3), color='blue', label='healpy')
for neighbour_tile_id in hp.get_all_neighbours(nside, *lonlat, nest=True, lonlat=True):
    if neighbour_tile_id > 0:
        plot(*hp_boundaries(neighbour_tile_id), color='black', lw=0.5, alpha=0.5, label=None)
        
top = np.radians(hp_boundaries_side_latlon(tile_id, 3))
bottom = np.radians(hp_boundaries_side_latlon(tile_id, 1))

distmat = haversine_distances(top.T, bottom.T)
top_i, bottom_i = np.unravel_index(distmat.argmin(), distmat.shape)

top_coord = top.T[top_i]
bottom_coord = bottom.T[bottom_i]

ax_cyl.scatter([np.degrees(top_coord[1])], [np.sin(top_coord[0])], color='green', marker='+')
ax_cyl.scatter([np.degrees(bottom_coord[1])], [np.sin(bottom_coord[0])], color='green', marker='+')
np.min(distmat)

In [ ]:
epsilon = 0.0000001

In [ ]:
tile_id

In [ ]:
hp.ang2pix(nside, np.degrees(top_coord[1]), np.degrees(top_coord[0]+epsilon), lonlat=True, nest=True)

In [ ]:
hp.ang2pix(nside, np.degrees(bottom_coord[1]+epsilon), np.degrees(bottom_coord[0]-epsilon), lonlat=True, nest=True)

In [ ]:
haversine_distances(np.array([[top_coord[0]+epsilon, top_coord[1]],]), np.array([[bottom_coord[0]-epsilon, bottom_coord[1]+epsilon],]))[0][0]